In [1]:
# pip install scrapy
# pip install --upgrade zope2

from collections import Counter
import foursquare
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import random
import requests
from scrapy import Selector
from scrapy.http import HtmlResponse
import seaborn as sns
from sqlalchemy import create_engine
import sqlite3
import textacy
import textacy.data
import time
import unicodedata

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
usda = pd.read_pickle('../../projects/Capstone Stuff/usda.pkl')
food = pd.read_pickle('../Capstone Stuff/explore_endpoint_df.pkl')
vegan = food[food.vegan_venue_check == 'vegan']

In [3]:
keyword = 'bakery'
if keyword.lower() in ', '.join(usda.Long_Desc.values).lower():
    print "true"

true


In [5]:
usda.head()

,NDB_No,FdGrp_Desc,Long_Desc,Nutr_No,Nutr_Val_per_100,Units,NutrDesc,Tagname,Not_Imputed,ComName,FdGrp_Cd
0,01001,Dairy and Egg Products,"Butter, salted",203,0.85,g,Protein,PROCNT,1,,0100
1,01001,Dairy and Egg Products,"Butter, salted",204,81.11,g,Total lipid (fat),FAT,1,,0100
2,01001,Dairy and Egg Products,"Butter, salted",205,0.06,g,"Carbohydrate, by difference",CHOCDF,0,,0100
3,01001,Dairy and Egg Products,"Butter, salted",207,2.11,g,Ash,ASH,1,,0100
4,01001,Dairy and Egg Products,"Butter, salted",208,717.00,kcal,Energy,ENERC_KCAL,0,,0100


In [8]:
usda = usda[['NDB_No','FdGrp_Desc','Long_Desc','Nutr_No','NutrDesc',
             'Nutr_Val_per_100','Units','Tagname','Not_Imputed']]

In [10]:
usda.head()

,NDB_No,FdGrp_Desc,Long_Desc,Nutr_No,NutrDesc,Nutr_Val_per_100,Units,Tagname,Not_Imputed
0,01001,Dairy and Egg Products,"Butter, salted",203,Protein,0.85,g,PROCNT,1
1,01001,Dairy and Egg Products,"Butter, salted",204,Total lipid (fat),81.11,g,FAT,1
2,01001,Dairy and Egg Products,"Butter, salted",205,"Carbohydrate, by difference",0.06,g,CHOCDF,0
3,01001,Dairy and Egg Products,"Butter, salted",207,Ash,2.11,g,ASH,1
4,01001,Dairy and Egg Products,"Butter, salted",208,Energy,717.00,kcal,ENERC_KCAL,0


In [11]:
usda.shape

(595359, 9)

In [39]:
tmp = usda[['FdGrp_Desc','NutrDesc']]

In [40]:
tmp = tmp.groupby(['FdGrp_Desc','NutrDesc']).agg(lambda x: 1.)
tmp.head()

FdGrp_Desc                           NutrDesc
American Indian/Alaska Native Foods  10:0        1.0
                                     12:0        1.0
                                     14:0        1.0
                                     14:1        1.0
                                     15:0        1.0
dtype: float64

In [41]:
tmp = tmp.unstack(fill_value=0)

In [42]:
tmp.shape

(25, 145)

In [43]:
tmp = tmp.reset_index()

In [44]:
tmp.head(10)

NutrDesc,FdGrp_Desc,10:0,12:0,13:0,14:0,14:1,15:0,15:1,16:0,16:1 c,...,"Vitamin C, total ascorbic acid",Vitamin D,Vitamin D (D2 + D3),Vitamin D2 (ergocalciferol),Vitamin D3 (cholecalciferol),Vitamin E (alpha-tocopherol),"Vitamin E, added",Vitamin K (phylloquinone),Water,"Zinc, Zn"
0,American Indian/Alaska Native Foods,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1,Baby Foods,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,Baked Products,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Beef Products,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Beverages,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
5,Breakfast Cereals,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,Cereal Grains and Pasta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
7,Dairy and Egg Products,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,Fast Foods,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
9,Fats and Oils,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
